<a href="https://colab.research.google.com/github/ShaunakSen/Data-Science-and-Machine-Learning/blob/master/Machine_Learning_Interpretibility.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## A Unique Method for Machine Learning Interpretability: Game Theory & Shapley Values!

[article link by ANKIT CHOUDHARY](https://www.analyticsvidhya.com/blog/2019/11/shapley-value-machine-learning-interpretability-game-theory/?utm_source=feedburner&utm_medium=email&utm_campaign=Feed%3A+AnalyticsVidhya+%28Analytics+Vidhya%29)

### What Is Game Theory

The key pioneers of game theory were mathematicians John von Neumann and John Nash, as well as economist Oskar Morgenstern.

Now, you might be asking – what is a game? Is it like chess? Video Games?

A “Game” is any situation in which there are several decision-makers, and each of them wants to optimize their results. The optimizing decision will depend on the decisions of others. The game identifies the players’ identities, preferences, and available strategies and how these strategies affect the outcome.

Game Theory attempts to define these situations in mathematical terms and determine what would happen if every player acts rationally.

- Perhaps an equilibrium can be reached (which is why we all drive on the same side of the road within a country)
- Maybe this equilibrium will be worse for all players (which is why people litter or pollute common resources)
- Well, everyone will try to be as unpredictable as possible in their actions (as might happen with troop deployment in a war)


In essence, Game Theory is a way to mathematically model complex human behavior, to try to understand it, and predict it.

### Cooperative Game Theory

> Cooperative game theory assumes that groups of players, called coalitions, are the primary units of decision-making, and may enforce cooperative behavior.


Consequently, cooperative games can be seen as a competition between coalitions of players, rather than between individual players.



### The Intuition behind Shapley Values


Let’s first design a cooperative game. Three friends – Ram, Abhiraj, and Pranav – go out for a meal. They order and share fries, wine, and pi. It is hard to figure out who should pay how much since they did not eat an equal share. So, we have the following information:

- If Ram is eating alone, he would pay 800
- If Abhiraj is eating alone, he would pay 560
- If Pranav is eating alone, he would pay 700
- If Ram and Abhiraj both eat alone, they would pay 800
- If Ram and Pranav both eat alone, they would pay 850
- If Abhiraj and Pranav both eat alone, they would pay 720
- If Ram, Abhiraj, and Pranav all eat together, they would pay 900

So, it turns out the actual amount all 3 of them pay when they eat together is 900. Now, the task at hand is to figure out how much each of them should pay individually.

The method we will adapt here is:

> We take all permutations of the 3 participants in sequence and see the incremental payout that each of them has to make.

So here, the sequence is Ram, Abhiraj and then Pranav in turn. As described above, Ram comes and pays 800. Now, Ram and Abhiraj pay 800 only so there is no additional payout for Abhiraj. Hence, we get 0. And finally, all 3 eat together and pay 900 so the additional payout for Pranav is 100.

We repeat the same exercise for each possible order for the 3 friends and get the following marginal payout values:

- (Ram, Abhiraj, Pranav) – (800,0,100)
- (Abhiraj, Ram, Pranav) – (560, 240, 100)
- (Abhiraj, Pranav, Ram) – (560, 160, 180)
- (Pranav, Ram, Abhiraj) – (700, 150, 50)
- (Pranav, Abhiraj, Ram) – (700, 20, 180)
- (Ram, Pranav, Abhiraj) – (800, 50, 50)

**So, what is the Shapley value for Ram, Abhiraj, and Pranav each? It is just the average of the marginal payout for each!**

For example, for Ram it is (800 + 240 + 180 + 150 + 180 + 800)/6 = 392. Similarly, for Abhiraj it is 207, and for Pranav, it turns out to be 303. The total turns out to be 900.

So now we have reached to the final amount that each of them should pay if all 3 go out together. In the next section, we will see how we can use the concept of Shapley values to interpret machine learning models.